Notebook for testing random things

In [11]:
from cmaqpy.runcmaq import CMAQModel
import datetime

In [8]:
# Specify the start/end times
start_datetime = 'August 05, 2016'  # first day that you want for MCIP/CCTM
end_datetime = 'August 15, 2016'  # add one day to the final day that you want to run MCIP/CCTM
# Specify if you want to run the 12 km or the 4 km domain
appl = '2016_12OTC2'
# appl = '2016_4OTC2'
# Define the coordinate name (must match that in GRIDDESC)
coord_name = 'LAM_40N97W'
if appl == '2016_12OTC2':
    grid_name = '12OTC2'
elif appl == '2016_4OTC2':
    grid_name = '4OTC2'

# Create a CMAQModel object
cmaq_sim = CMAQModel(start_datetime, end_datetime, appl, coord_name, grid_name, verbose=True)

Application name: 2016_12OTC2
Coordinate name: LAM_40N97W
Grid name: 12OTC2
CMAQ run starting on: 2016-08-05 00:00:00
CMAQ run ending on: 2016-08-15 00:00:00


In [10]:
cmaq_sim.delt.days

10

In [15]:
start_datetimes_lst = [single_date for single_date in (cmaq_sim.start_datetime + datetime.timedelta(n) for n in range(cmaq_sim.delt.days))]
end_datetimes_lst = [single_date for single_date in (cmaq_sim.start_datetime + datetime.timedelta(n + 1) for n in range(cmaq_sim.delt.days))]

In [17]:
for day_no in range(cmaq_sim.delt.days):
    start_datetime = cmaq_sim.start_datetime + datetime.timedelta(day_no)
    end_datetime = cmaq_sim.start_datetime + datetime.timedelta(day_no + 1)
    print(day_no)
    print(start_datetime)
    print(end_datetime)

0
2016-08-05 00:00:00
2016-08-06 00:00:00
1
2016-08-06 00:00:00
2016-08-07 00:00:00
2
2016-08-07 00:00:00
2016-08-08 00:00:00
3
2016-08-08 00:00:00
2016-08-09 00:00:00
4
2016-08-09 00:00:00
2016-08-10 00:00:00
5
2016-08-10 00:00:00
2016-08-11 00:00:00
6
2016-08-11 00:00:00
2016-08-12 00:00:00
7
2016-08-12 00:00:00
2016-08-13 00:00:00
8
2016-08-13 00:00:00
2016-08-14 00:00:00
9
2016-08-14 00:00:00
2016-08-15 00:00:00
